In [3]:
from google.colab import files
uploaded = files.upload()


Saving dataset.zip to dataset.zip


In [4]:
import zipfile
import os

zip_path = "dataset.zip"  # <-- change if needed
extract_path = "/content/dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("ZIP extracted successfully!")


ZIP extracted successfully!


In [6]:
import os
import shutil
import random

base_path = "/content/dataset/dataset/train"  # your current dataset
new_base = "/content/dataset"         # where new folders will be created

classes = os.listdir(base_path)

for split in ["train", "val", "test"]:
    for c in classes:
        os.makedirs(f"{new_base}/{split}/{c}", exist_ok=True)

for c in classes:
    img_dir = f"{base_path}/{c}"
    images = os.listdir(img_dir)

    random.shuffle(images)

    train_split = int(0.8 * len(images))
    val_split   = int(0.9 * len(images))

    train_imgs = images[:train_split]
    val_imgs   = images[train_split:val_split]
    test_imgs  = images[val_split:]

    # Move files
    for img in train_imgs:
        shutil.copy(f"{img_dir}/{img}", f"{new_base}/train/{c}/{img}")
    for img in val_imgs:
        shutil.copy(f"{img_dir}/{img}", f"{new_base}/val/{c}/{img}")
    for img in test_imgs:
        shutil.copy(f"{img_dir}/{img}", f"{new_base}/test/{c}/{img}")

print("Dataset Successfully Split into train, val, and test.")

Dataset Successfully Split into train, val, and test.


In [7]:
!tree /content/dataset -L 3


/bin/bash: line 1: tree: command not found


In [8]:
!pip install ultralytics
from ultralytics import YOLO

model = YOLO("yolov8n-cls.pt")  # classification model

model.train(
    data="/content/dataset",
    epochs=200,
    imgsz=224,
    batch=32,
    lr0=0.001,
    optimizer="Adam",
    dropout=0.2,
    augment=True,   # enable augmentations
    mixup=0.3,
    cutmix=0.3,
    erasing=0.4,
)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.9 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.3, data=/content/dataset, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.2, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0

ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d5ec4fad430>
curves: []
curves_results: []
fitness: 0.800000011920929
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.6000000238418579, 'metrics/accuracy_top5': 1.0, 'fitness': 0.800000011920929}
save_dir: PosixPath('/content/runs/classify/train')
speed: {'preprocess': 0.3271780666712706, 'inference': 0.8352838666648191, 'loss': 0.0007405333311301849, 'postprocess': 0.0010631333376901844}
task: 'classify'
top1: 0.6000000238418579
top5: 1.0

In [10]:
model = YOLO("/content/runs/classify/train/weights/best.pt")
model.predict("/content/dataset/test/rating_2/low_quality_33.jpg")


image 1/1 /content/dataset/test/rating_2/low_quality_33.jpg: 224x224 rating_2 0.58, rating_1 0.41, rating_3 0.01, rating_5 0.00, rating_4 0.00, 4.6ms
Speed: 13.9ms preprocess, 4.6ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: None
 keypoints: None
 masks: None
 names: {0: 'rating_1', 1: 'rating_2', 2: 'rating_3', 3: 'rating_4', 4: 'rating_5'}
 obb: None
 orig_img: array([[[25,  4, 13],
         [26,  5, 14],
         [27,  6, 15],
         ...,
         [25,  4, 13],
         [27,  6, 15],
         [28,  7, 16]],
 
        [[25,  4, 13],
         [26,  5, 14],
         [27,  6, 15],
         ...,
         [25,  4, 13],
         [26,  5, 14],
         [27,  6, 15]],
 
        [[25,  4, 13],
         [26,  5, 14],
         [27,  6, 15],
         ...,
         [24,  3, 12],
         [25,  4, 13],
         [26,  5, 14]],
 
        ...,
 
        [[28,  7, 16],
         [27,  6, 15],
         [25,  4, 13],
         ...,
         [24,  3, 12],
         [25,  4, 13],
         [26,  5, 14]],
 
        [[28,  7, 16],
         [27,  6, 15],
         [25,  4, 13],
         ...,
         [24,  3, 12],
         [24,  3, 12],
         [24,  3, 12]],
 


In [11]:
model.export(format="onnx")


Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/

PyTorch: starting from '/content/runs/classify/train/weights/best.pt' with input shape (1, 3, 224, 224) BCHW and output shape(s) (1, 5) (2.8 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<=1.19.1', 'onnxslim>=0.1.71', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 14 packages in 182ms
Prepared 6 packages in 7.42s
Installed 6 packages in 238ms
 + colorama==0.4.6
 + coloredlogs==15.0.1
 + humanfriendly==10.0
 + onnx==1.19.1
 + onnxruntime-gpu==1.23.2
 + onnxslim==0.1.74

requirements: AutoUpdate success ✅ 8.7s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.19.1 opset 22...
ONNX: slimming with onnxslim 0.1.74...
ONNX: exp

'/content/runs/classify/train/weights/best.onnx'

In [12]:
from ultralytics import YOLO

model = YOLO("/content/runs/classify/train/weights/best.pt")


In [33]:
from google.colab import files
uploaded = files.upload()


Saving image_1.jpg to image_1.jpg


In [34]:
results = model.predict("image_1.jpg")



image 1/1 /content/image_1.jpg: 224x224 rating_4 0.99, rating_5 0.01, rating_3 0.00, rating_2 0.00, rating_1 0.00, 8.0ms
Speed: 56.4ms preprocess, 8.0ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)


In [35]:
for r in results:
    print("Predicted class:", r.names[r.probs.top1])
    print("Confidence:", float(r.probs.top1conf))


Predicted class: rating_4
Confidence: 0.9880346655845642
